In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
from binance import Client
from datetime import datetime 

import src.utils_binance as utils
import src.utils as utils2
import src.myenv as myenv
import time
myenv.telegram_key.append(utils2.get_telegram_key())

import numpy as np
import pandas as pd


In [ ]:
utils.get_account_balance()

In [ ]:
#client.get_account_snapshot(type='SPOT')

In [ ]:
symbol = 'LINKUSDT'
symbol_info = utils.get_client().get_symbol_info(symbol=symbol)
print(f'symbol_info: {symbol_info}')


In [ ]:
price_order = utils.get_client().get_symbol_ticker(symbol=symbol)
symbol_info, symbol_precision, quote_precision, quantity_precision, price_precision, step_size, tick_size = utils.get_symbol_info(symbol)
price = round(float(price_order['price']), symbol_precision)

take_profit, stop_loss = utils.calc_take_profit_stop_loss('LONG', price, 1.0, 1)
print((symbol_precision, price_precision, step_size, tick_size))
print((f'price: {price} - take_profit: {take_profit} - stop_loss: {stop_loss}'))

In [ ]:
amount_invested, balance = utils.get_amount_to_invest()  # ok

ledger_params = utils.get_params_operation(np.datetime64('now'), symbol, '1m', 'BUY', 1.0, amount_invested, take_profit, stop_loss,
												price, 30.0, take_profit, 0.0, 1.0, 'LONG', balance, symbol_precision, quote_precision, quantity_precision, price_precision, tick_size, step_size)  # ok
print(ledger_params)

In [ ]:
#order_buy_id, order_sell_id = utils.register_operation(ledger_params)
#print((order_buy_id, order_sell_id))

In [ ]:
symbol = 'SOLUSDT'
interval = '15m'

id_buy = f'{symbol}_{interval}_buy'
id_limit = f'{symbol}_{interval}_limit'
id_stop = f'{symbol}_{interval}_stop'
orders = utils.get_client().get_all_orders(symbol=symbol)
orders

In [16]:
symbol = 'SOLUSDT'
interval = '1m'

id_limit = f'{symbol}_{interval}_limit'

order = utils.get_client().get_order(symbol=symbol, origClientOrderId=id_limit)
order

{'symbol': 'SOLUSDT',
 'orderId': 4637803346,
 'orderListId': 96272979,
 'clientOrderId': 'SOLUSDT_1m_limit',
 'price': '113.03000000',
 'origQty': '0.29000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT_MAKER',
 'side': 'SELL',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1703834178870,
 'updateTime': 1703834178870,
 'isWorking': True,
 'workingTime': 1703834178870,
 'origQuoteOrderQty': '0.00000000',
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [ ]:
purshased_price = 0.0
stop_loss = 0.0
take_profit = 0.0
df_order = pd.DataFrame(orders)
key = (df_order['clientOrderId'] == id_buy) | (df_order['clientOrderId'] == id_limit) | (df_order['clientOrderId'] == id_stop)
if True: # key.sum() > 0:
	df_order = df_order[key]
	is_purchased = df_order['status'].isin([Client.ORDER_STATUS_NEW, Client.ORDER_STATUS_PARTIALLY_FILLED, Client.ORDER_STATUS_PENDING_CANCEL]).sum() > 0
	if is_purchased:
		has_buy = df_order['clientOrderId'] == id_buy
		if has_buy.sum() > 0:
			purshased_price = float(df_order[has_buy].tail(1)['price'].values[0])
		has_limit = df_order['clientOrderId'] == id_limit
		if has_limit.sum() > 0:
			take_profit = float(df_order[has_limit].tail(1)['price'].values[0])
		has_stop = df_order['clientOrderId'] == id_stop
		if has_stop.sum() > 0:
			stop_loss = float(df_order[has_stop].tail(1)['price'].values[0])

print(f'purshased_price: {purshased_price} - stop_loss: {stop_loss} - take_profit: {take_profit}')
#df_order.info()
df_order

In [ ]:
purchased, purchase_price, amount_invested, take_profit, stop_loss = utils.is_purchased(symbol, interval)

print((f'symbol: {symbol} - interval: {interval} - purchased: {purchased} - purchase_price: {purchase_price} - amount_invested: {amount_invested} - take_profit: {take_profit} - stop_loss: {stop_loss}'))

In [ ]:
symbol_info, symbol_precision, quote_precision, quantity_precision, price_precision, step_size, tick_size = utils.get_symbol_info(symbol)

_time = pd.DataFrame(data=[utils.get_client().get_server_time()])				 

take_profit, stop_loss = utils.calc_take_profit_stop_loss('LONG', 3.048, 4.5, 2) 

ledger_params = utils.get_params_operation(_time['serverTime'].values[0], symbol, interval, 'BUY', 4.5, 10.0,
											take_profit, stop_loss, 3.048, 0, take_profit, 0.0, 0.0, 'LONG', balance,
											symbol_precision, quote_precision, quantity_precision, price_precision, step_size, tick_size)  # ok
print(ledger_params)

In [ ]:
utils.register_operation(ledger_params)